In [2]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tensorflow.python.framework import ops

vgg16
<img src="img/vgg16.png">

fcn-vgg16
<img src="img/fcn_vgg.png">

In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    # load the graph
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    
    # point to default graph
    graph = tf.get_default_graph()
    # grab each layer (tensor) by name
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

In [3]:
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [13]:
??tf.layers.conv2d

tf.layers.conv2d
def conv2d(inputs,
           filters,
           kernel_size,
           strides=(1, 1),
           padding='valid',
           data_format='channels_last',
           dilation_rate=(1, 1),
           activation=None,
           use_bias=True,
           kernel_initializer=None,
           bias_initializer=init_ops.zeros_initializer(),
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           trainable=True,
           name=None,
           reuse=None):

In [15]:
??tf.layers.conv2d_transpose
"""
def conv2d_transpose(inputs,
                     filters,
                     kernel_size,
                     strides=(1, 1),
                     padding='valid',
                     data_format='channels_last',
                     activation=None,
                     use_bias=True,
                     kernel_initializer=None,
                     bias_initializer=init_ops.zeros_initializer(),
                     kernel_regularizer=None,
                     bias_regularizer=None,
                     activity_regularizer=None,
                     trainable=True,
                     name=None,
                     reuse=None):
"""

In [4]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # 1x1 convolution of vgg layer 7, using L2 regularizer
    layer7a_out = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 
                                   padding= 'same', 
                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    # upsample by 2 using conv2d_transpose, strides value mainly determined the umsampling factor.
    layer4a_in1 = tf.layers.conv2d_transpose(layer7a_out, num_classes, 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # make sure the shapes are the same!
    # 1x1 convolution of vgg layer 4
    layer4a_in2 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 
                                   padding= 'same', 
                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # skip connection (element-wise addition)
    layer4a_out = tf.add(layer4a_in1, layer4a_in2)
    # upsample by 2 (2x2=4)
    layer3a_in1 = tf.layers.conv2d_transpose(layer4a_out, num_classes, 4,  
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # 1x1 convolution of vgg layer 3
    layer3a_in2 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 
                                   padding= 'same', 
                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # skip connection (element-wise addition)
    layer3a_out = tf.add(layer3a_in1, layer3a_in2)
    # upsample (2x2x2=8)
    # nn_last_layer: last layer of fcn, it is logits
    nn_last_layer = tf.layers.conv2d_transpose(layer3a_out, num_classes, 16,  
                                               strides= (8, 8), 
                                               padding= 'same', 
                                               kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                               kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    return nn_last_layer

In [4]:
# debug usage
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # 1x1 convolution of vgg layer 7, using L2 regularizer
    layer7a_out = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 
                                   padding= 'same', 
                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01),
                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    # upsample by 2 using conv2d_transpose, strides value mainly determined the umsampling factor.
    layer4a_in1 = tf.layers.conv2d_transpose(layer7a_out, num_classes, 4, 
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # make sure the shapes are the same!
    # 1x1 convolution of vgg layer 4
    layer4a_in2 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 
                                   padding= 'same', 
                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # skip connection (element-wise addition)
    layer4a_out = tf.add(layer4a_in1, layer4a_in2)
    # upsample by 2 (2x2=4)
    layer3a_in1 = tf.layers.conv2d_transpose(layer4a_out, num_classes, 4,  
                                             strides= (2, 2), 
                                             padding= 'same', 
                                             kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                             kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # 1x1 convolution of vgg layer 3
    layer3a_in2 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 
                                   padding= 'same', 
                                   kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                   kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    # skip connection (element-wise addition)
    layer3a_out = tf.add(layer3a_in1, layer3a_in2)
    # upsample (2x2x2=8)
    # nn_last_layer: last layer of fcn, it is logits
    nn_last_layer = tf.layers.conv2d_transpose(layer3a_out, num_classes, 16,  
                                               strides= (8, 8), 
                                               padding= 'same', 
                                               kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                               kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    
    # debug the dimention of last layer
    # tf.shape() will print the shape after the node is running
    tf.Print(nn_last_layer, [tf.shape(nn_last_layer)])
    # tf.Print(nn_last_layer, [tf.shape(nn_last_layer)[1:3]]) # only to view the height and width.
    return nn_last_layer

In [5]:
tests.test_layers(layers)

Tests Passed


In [5]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    # make logits a 2D tensor where each row represents a pixel and each column a class
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1,num_classes))
    # define loss function
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label))
    # define training operation
    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)

    return logits, train_op, cross_entropy_loss

In [7]:
tests.test_optimize(optimize)

Tests Passed


In [6]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate       
    
    """
    
    # Create a saver instance
    saver = tf.train.Saver()
    # Specify the location to save the model
    DIR = "./saves"
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    print()
    for i in range(epochs):
        print("EPOCH {} ...".format(i+1))
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], 
                               feed_dict={input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 0.0009})
            print("Loss: = {:.3f}".format(loss))
        # save model for each 10 epochs
        if i % 10 ==0 :
            saver.save(sess, os.path.join(DIR, "nn_model"), global_step=i)
        if i == epochs - 1:
            saver.save(sess, os.path.join(DIR, "nn_model"), global_step=i)
        print()

In [9]:
tests.test_train_nn(train_nn)

In [7]:
def run():
    # to be able to rerun the model without overwriting tf variables
    ops.reset_default_graph()    
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        # data_dir = './data'
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
        # nn_last_layer: final layers of the fcnn, it is logits (score)
        nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)

        epochs = 50
        batch_size = 5

        # TF placeholders
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
       

        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function

        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video (refernece from previoous advanced lane detection project)

In [10]:
num_classes = 2
image_shape = (160, 576)
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)

with tf.Session() as sess:
    helper.save_inference_training_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

Tests Passed


NameError: name 'logits' is not defined

In [8]:
if __name__ == '__main__':
    run()

Tests Passed
Training...

EPOCH 1 ...
Loss: = 2.078
Loss: = 8.761
Loss: = 1.053
Loss: = 0.847
Loss: = 0.881
Loss: = 0.833
Loss: = 0.737
Loss: = 0.667
Loss: = 0.741
Loss: = 0.739
Loss: = 0.648
Loss: = 0.661
Loss: = 0.683
Loss: = 0.695
Loss: = 0.642
Loss: = 0.625
Loss: = 0.610
Loss: = 0.624
Loss: = 0.606
Loss: = 0.604
Loss: = 0.556
Loss: = 0.565
Loss: = 0.544
Loss: = 0.517
Loss: = 0.596
Loss: = 0.519
Loss: = 0.540
Loss: = 0.472
Loss: = 0.491
Loss: = 0.434
Loss: = 0.459
Loss: = 0.492
Loss: = 0.371
Loss: = 0.457
Loss: = 0.411
Loss: = 0.386
Loss: = 0.351
Loss: = 0.335
Loss: = 0.464
Loss: = 0.330
Loss: = 0.289
Loss: = 0.355
Loss: = 0.307
Loss: = 0.314
Loss: = 0.273
Loss: = 0.258
Loss: = 0.274
Loss: = 0.230
Loss: = 0.270
Loss: = 0.251
Loss: = 0.257
Loss: = 0.263
Loss: = 0.321
Loss: = 0.297
Loss: = 0.215
Loss: = 0.246
Loss: = 0.260
Loss: = 0.311

EPOCH 2 ...
Loss: = 0.186
Loss: = 0.395
Loss: = 0.296
Loss: = 0.236
Loss: = 0.298
Loss: = 0.261
Loss: = 0.571
Loss: = 0.237
Loss: = 0.276
Loss: = 0.2

Loss: = 0.143
Loss: = 0.098
Loss: = 0.087
Loss: = 0.063
Loss: = 0.083

EPOCH 11 ...
Loss: = 0.070
Loss: = 0.110
Loss: = 0.069
Loss: = 0.051
Loss: = 0.081
Loss: = 0.105
Loss: = 0.090
Loss: = 0.079
Loss: = 0.074
Loss: = 0.085
Loss: = 0.074
Loss: = 0.062
Loss: = 0.093
Loss: = 0.090
Loss: = 0.067
Loss: = 0.108
Loss: = 0.111
Loss: = 0.068
Loss: = 0.081
Loss: = 0.079
Loss: = 0.119
Loss: = 0.052
Loss: = 0.088
Loss: = 0.085
Loss: = 0.079
Loss: = 0.066
Loss: = 0.092
Loss: = 0.114
Loss: = 0.064
Loss: = 0.097
Loss: = 0.078
Loss: = 0.086
Loss: = 0.101
Loss: = 0.071
Loss: = 0.070
Loss: = 0.066
Loss: = 0.069
Loss: = 0.068
Loss: = 0.049
Loss: = 0.090
Loss: = 0.086
Loss: = 0.125
Loss: = 0.070
Loss: = 0.117
Loss: = 0.054
Loss: = 0.046
Loss: = 0.109
Loss: = 0.075
Loss: = 0.078
Loss: = 0.107
Loss: = 0.107
Loss: = 0.063
Loss: = 0.091
Loss: = 0.075
Loss: = 0.082
Loss: = 0.068
Loss: = 0.058
Loss: = 0.138

EPOCH 12 ...
Loss: = 0.107
Loss: = 0.076
Loss: = 0.090
Loss: = 0.080
Loss: = 0.111
Loss: = 0.053
Loss: 

Loss: = 0.063
Loss: = 0.046
Loss: = 0.049
Loss: = 0.055
Loss: = 0.114
Loss: = 0.063
Loss: = 0.043
Loss: = 0.050
Loss: = 0.071

EPOCH 21 ...
Loss: = 0.043
Loss: = 0.079
Loss: = 0.039
Loss: = 0.049
Loss: = 0.047
Loss: = 0.060
Loss: = 0.099
Loss: = 0.055
Loss: = 0.037
Loss: = 0.049
Loss: = 0.056
Loss: = 0.059
Loss: = 0.039
Loss: = 0.063
Loss: = 0.047
Loss: = 0.050
Loss: = 0.047
Loss: = 0.063
Loss: = 0.091
Loss: = 0.035
Loss: = 0.047
Loss: = 0.062
Loss: = 0.047
Loss: = 0.045
Loss: = 0.048
Loss: = 0.044
Loss: = 0.052
Loss: = 0.076
Loss: = 0.041
Loss: = 0.064
Loss: = 0.062
Loss: = 0.077
Loss: = 0.041
Loss: = 0.064
Loss: = 0.042
Loss: = 0.042
Loss: = 0.064
Loss: = 0.063
Loss: = 0.035
Loss: = 0.052
Loss: = 0.078
Loss: = 0.052
Loss: = 0.056
Loss: = 0.042
Loss: = 0.046
Loss: = 0.054
Loss: = 0.042
Loss: = 0.041
Loss: = 0.043
Loss: = 0.050
Loss: = 0.059
Loss: = 0.050
Loss: = 0.068
Loss: = 0.047
Loss: = 0.066
Loss: = 0.065
Loss: = 0.042
Loss: = 0.054

EPOCH 22 ...
Loss: = 0.060
Loss: = 0.062
Loss: 

Loss: = 0.064
Loss: = 0.038
Loss: = 0.052
Loss: = 0.051
Loss: = 0.031
Loss: = 0.050
Loss: = 0.029
Loss: = 0.044
Loss: = 0.056
Loss: = 0.056
Loss: = 0.057
Loss: = 0.049
Loss: = 0.044

EPOCH 31 ...
Loss: = 0.055
Loss: = 0.074
Loss: = 0.034
Loss: = 0.050
Loss: = 0.047
Loss: = 0.049
Loss: = 0.053
Loss: = 0.067
Loss: = 0.049
Loss: = 0.047
Loss: = 0.043
Loss: = 0.046
Loss: = 0.032
Loss: = 0.050
Loss: = 0.034
Loss: = 0.044
Loss: = 0.053
Loss: = 0.048
Loss: = 0.040
Loss: = 0.033
Loss: = 0.026
Loss: = 0.054
Loss: = 0.038
Loss: = 0.029
Loss: = 0.041
Loss: = 0.035
Loss: = 0.044
Loss: = 0.040
Loss: = 0.035
Loss: = 0.045
Loss: = 0.028
Loss: = 0.038
Loss: = 0.044
Loss: = 0.035
Loss: = 0.033
Loss: = 0.049
Loss: = 0.038
Loss: = 0.027
Loss: = 0.028
Loss: = 0.041
Loss: = 0.046
Loss: = 0.045
Loss: = 0.058
Loss: = 0.043
Loss: = 0.038
Loss: = 0.036
Loss: = 0.035
Loss: = 0.033
Loss: = 0.042
Loss: = 0.042
Loss: = 0.031
Loss: = 0.027
Loss: = 0.033
Loss: = 0.045
Loss: = 0.030
Loss: = 0.028
Loss: = 0.035
Loss: 

Loss: = 0.016
Loss: = 0.017
Loss: = 0.027
Loss: = 0.031
Loss: = 0.027
Loss: = 0.025
Loss: = 0.028
Loss: = 0.034
Loss: = 0.028
Loss: = 0.025
Loss: = 0.032
Loss: = 0.026
Loss: = 0.022
Loss: = 0.022
Loss: = 0.016
Loss: = 0.020
Loss: = 0.037

EPOCH 41 ...
Loss: = 0.022
Loss: = 0.027
Loss: = 0.025
Loss: = 0.021
Loss: = 0.028
Loss: = 0.031
Loss: = 0.028
Loss: = 0.025
Loss: = 0.017
Loss: = 0.013
Loss: = 0.032
Loss: = 0.041
Loss: = 0.031
Loss: = 0.027
Loss: = 0.037
Loss: = 0.028
Loss: = 0.027
Loss: = 0.020
Loss: = 0.025
Loss: = 0.021
Loss: = 0.020
Loss: = 0.022
Loss: = 0.027
Loss: = 0.021
Loss: = 0.027
Loss: = 0.027
Loss: = 0.014
Loss: = 0.022
Loss: = 0.031
Loss: = 0.030
Loss: = 0.019
Loss: = 0.019
Loss: = 0.020
Loss: = 0.029
Loss: = 0.018
Loss: = 0.027
Loss: = 0.025
Loss: = 0.026
Loss: = 0.031
Loss: = 0.032
Loss: = 0.025
Loss: = 0.024
Loss: = 0.027
Loss: = 0.024
Loss: = 0.018
Loss: = 0.035
Loss: = 0.012
Loss: = 0.025
Loss: = 0.017
Loss: = 0.033
Loss: = 0.023
Loss: = 0.027
Loss: = 0.018
Loss: 

Loss: = 0.019
Loss: = 0.031
Loss: = 0.027
Loss: = 0.013
Loss: = 0.022
Loss: = 0.016
Loss: = 0.021
Loss: = 0.021
Loss: = 0.022
Loss: = 0.023
Loss: = 0.029
Loss: = 0.022
Loss: = 0.026
Loss: = 0.020
Loss: = 0.020
Loss: = 0.019
Loss: = 0.017
Loss: = 0.016
Loss: = 0.023
Loss: = 0.015
Loss: = 0.016

Training Finished. Saving test images to: ./runs/1508746462.9806385


In [ ]:
# load from saved checkpoint file and perform inference on certain images
# Restore from the trained model
#tf.reset_default_graph()

# Create a saver instance
saver = tf.train.Saver()
# Specify the location to save the model
DIR = "/root/tmp/tf_model"

with tf.Session() as sess:
    saver.restore(sess, os.path.join(DIR, "nn_model-300"))
    
    # Calculate the correct predictions
    correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

    # Calculate accuracy on the test set
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    #print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
    print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
